In [1]:
import sys, os

import torch
from torch import nn
import torchvision.models as models

import image_prep as prep
import image_utils as utils
import finetune_model as finetune
from image_gradient import RGBgradients
import activation_maximization as am

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_path = '../data'
images_path = '%s/architecture' % data_path
originals_path = '%s/orig_sized' % data_path
tensors_path = '%s/image_tensors' % data_path

## Resize and Crop the images

In [ ]:
im_sizes = prep.get_image_sizes(images_path)

prep.resize_and_crop(im_sizes, originals_path)

## Create image tensor files

In [ ]:
lab2idx = prep.get_lab_to_idx(images_path)

images_to_tensors(images_path, tensors_path, lab2idx, im_size=128, max_per_file=2000)

## Download the pretrained InceptionV3 model and finetune on architecture data

In [ ]:
model = models.quantization.inception_v3(pretrained=True, aux_logits=False)

#replace the classification layer
model.fc = nn.Linear(2048, len(lab2idx))

model

In [ ]:
model_path = 'inception_model'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model, train_losses, cv_losses = finetune.finetune_model(model, tensor_path, model_path, 
                                                         lab2idx, device, batch_size=8, n_epochs=50)

In [ ]:
#plot train loss

fig = plt.figure()
plt.plot(train_losses, color='blue')
plt.legend(['Train Loss'], loc='upper right')

In [ ]:
#plot cv loss

fig = plt.figure()
plt.plot(cv_losses, color='orange')
plt.legend(['CV Loss'], loc='upper right')

## Load best model and maximize activations of various neurons

In [ ]:
model.load_state_dict(torch.load('%s/model.pt' % model_path))
model.eval()

#freeze model params
for param in model.parameters():
    param.requires_grad_(False)

#view the layer choices
for n,p in model.named_parameters():
    if '.conv.' in n:
        print(p.size(), n)

In [ ]:
# We will register a forward hook to get the output of the layers
activation = {} 

def create_hook(name):
    def hook(m, i, o):
        # copy the output of the given layer
        activation[name] = o
    return hook

In [ ]:
#choose a layer
layer_name = 'Mixed_7a.branch3x3_1'

#create a hook for the layer
model.Mixed_7a.branch3x3_1.conv.register_forward_hook(create_hook(layer_name))

#choose a neuron
unit_idx = 5

#activations will be stored here, make sure it's clear
activation = {}

#create the pixel gradient calculating network
gradLayer = RGBgradients()

#call maximize_activation_with_scaling
am.maximize_activation_with_scaling(model, gradLayer, activation, layer_name, unit_idx,
                                    upscaling_steps=20, upscaling_factor=1.05, 
                                    optim_steps=40, optim_lr=0.4, act_wt=0.5,
                                    model_name='inception')

In [ ]:
#call maximize_activation_octaves
am.maximize_activation_octaves(model, gradLayer, activation, layer_name, unit_idx, im_size=(256,256), 
                               num_octaves=4, octave_scale=1.4, optim_steps=20, optim_lr=0.4, act_wt=0.75,
                               model_name='inception')

## Compare to original InceptionV3 model

In [ ]:
orig_model = models.quantization.inception_v3(pretrained=True, aux_logits=False)

orig_model.eval()

#freeze model params
for param in orig_model.parameters():
    param.requires_grad_(False)

In [ ]:
#choose a layer
layer_name = 'Mixed_7a.branch3x3_1'

#create a hook for the layer
orig_model.Mixed_7a.branch3x3_1.conv.register_forward_hook(create_hook(layer_name))

#choose a neuron
unit_idx = 5

#activations will be stored here, make sure it's clear
activation = {}

#create the pixel gradient calculating network
gradLayer = RGBgradients()

#call maximize_activation_with_scaling
am.maximize_activation_with_scaling(orig_model, gradLayer, activation, layer_name, unit_idx,
                                    upscaling_steps=20, upscaling_factor=1.05, 
                                    optim_steps=40, optim_lr=0.4, act_wt=0.5,
                                    model_name='inception')